In [1]:
import time
import gspread 
from oauth2client.service_account import ServiceAccountCredentials
from alpha_vantage.timeseries import TimeSeries
from pprint import pprint
import pandas as pd
import numpy as np
from ib.opt import Connection, message
from ib.ext.Contract import Contract
from ib.ext.Order import Order

Constant = 5
ClientID = 11479

def error_handler(msg):
    print("Server Error:", msg)

def server_handler(msg):
    print("Server Msg:", msg.typeName, "-", msg)

def make_contract(symbol, sec_type, exch, prim_exch, curr):
    Contract.m_symbol = symbol
    Contract.m_secType = sec_type
    Contract.m_exchange = exch
    Contract.m_primaryExch = prim_exch
    Contract.m_currency = curr
    return Contract

def make_order(action, quantity):
        order = Order()
        order.m_orderType = 'MKT'         
        order.m_totalQuantity = quantity
        order.m_action = action        
        return order

while True:
    scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
    credentials = ServiceAccountCredentials.from_json_keyfile_name('C:/Users/pavel/Downloads/TradingSignals-0afb3f18df64.json', scope)
    gc = gspread.authorize(credentials)
    wks = gc.open('Tradingsignals').sheet1   
    n = 2 
    
    while True:
        
        if wks.cell(n,1).value not in(''):
            Ticker = wks.cell(n,2).value
            Signal = wks.cell(n,3).value
            Quantity = int(wks.cell(n,4).value)
            ts = TimeSeries(key='H3YG4J5IUAE6CUSA', output_format='pandas')
            data60, meta_data = ts.get_intraday(symbol=wks.cell(n,2).value, interval='60min', outputsize='compact')
            data60.columns = ['open','high','low','close','vol']
            span = 20
            sma = data60.open.rolling(window=span, min_periods=20).mean()[:span]
            rest = data60.open[span:]
            ema20 = pd.concat([sma,rest]).ewm(span=span, adjust=False).mean()
            span1 = 50
            sma1 = data60.close.rolling(window=span1, min_periods=50).mean()[:span1]
            rest1 = data60.close[span1:]
            ema50 = pd.concat([sma1,rest1]).ewm(span=span1, adjust=False).mean()
                
            if wks.cell(n,3).value in('BUY'):
                print(wks.cell(n,2).value, wks.cell(n,3).value)
                print('Checking Trend...')
                
                if ema20[-1] > ema50[-2]:
                    data15, meta_data = ts.get_intraday(symbol=wks.cell(n,2).value, interval='15min', outputsize='compact')
                    data15.columns = ['open','high','low','close','vol']
                    print('Trend OK')
                    print('Checking Momentum...')
            
                    if data15.close[-1] < 1.005*(data15.close[-2]): 
                        data5, meta_data = ts.get_intraday(symbol=wks.cell(n,2).value, interval='5min', outputsize='full')
                        data5.columns = ['open','high','low','close','vol']
                        print('Momentum OK')
                        print('Checking Volume...')
                        
                        if data5.vol[-1]+data5.vol[-2] > 3*(np.mean(data5.vol[-224:-3])):
                            print('Volume OK')
                            print('Initiating Trasaction...')
                            
                            if __name__ == "__main__":
                                conn = Connection.create(port=7497, clientId=ClientID)
                                conn.connect()
                                conn.register(error_handler, 'Error')
                                conn.registerAll(server_handler)
                                oid = int(n)+Constant
                                cont = make_contract(Ticker, 'STK', 'SMART', 'SMART', 'USD')
                                offer = make_order(Signal, Quantity)
                                conn.placeOrder(oid, cont, offer)
                                conn.disconnect()
                                time.sleep(55)
            
            
            elif wks.cell(n,3).value in('SELL'):
                print(wks.cell(n,2).value, wks.cell(n,3).value)
                print('Checking Trend...')
               
                if ema20[-1] < ema50[-2]:
                    data15, meta_data = ts.get_intraday(symbol=wks.cell(n,2).value, interval='15min', outputsize='compact')
                    data15.columns = ['open','high','low','close','vol']
                    print('Trend OK')
                    print('Checking Momentum...')
            
                    if data15.close[-1] < 0.995*(data15.close[-2]): 
                        data5, meta_data = ts.get_intraday(symbol=wks.cell(n,2).value, interval='5min', outputsize='full')
                        data5.columns = ['open','high','low','close','vol']
                        print('Momentum OK')
                        print('Checking Volume...')
                        
                        if data5.vol[-1]+data5.vol[-2] > 3*(np.mean(data5.vol[-224:-3])):
                            print('Volume OK')
                            print('Initiating Trasaction...')
                            
                            if __name__ == "__main__":
                                conn = Connection.create(port=7497, clientId=ClientID)
                                conn.connect()
                                conn.register(error_handler, 'Error')
                                conn.registerAll(server_handler)
                                oid = int(n)+Constant
                                cont = make_contract(Ticker, 'STK', 'SMART', 'SMART', 'USD')
                                offer = make_order(Signal, Quantity)
                                conn.placeOrder(oid, cont, offer)
                                conn.disconnect()
                                time.sleep(55)
                
            n = n+1
        time.sleep(2)

AMZN BUY
Checking Trend...
Trend OK
Checking Momentum...
Momentum OK
Checking Volume...
Volume OK
Initiating Trasaction...
Server Version: 76
TWS Time at connection:20190315 12:09:24 Greenwich Mean Time


15-Mar-19 12:10:24 DEBUG     https://sheets.googleapis.com:443 "GET /v4/spreadsheets/1lF_AeqkipBki5AIdkDKokqWeB8HEGVrwIpQOWGFk1J8/values/sheet1%21A3?valueRenderOption=FORMATTED_VALUE HTTP/1.1" 200 None
15-Mar-19 12:10:25 DEBUG     https://sheets.googleapis.com:443 "GET /v4/spreadsheets/1lF_AeqkipBki5AIdkDKokqWeB8HEGVrwIpQOWGFk1J8/values/sheet1%21B3?valueRenderOption=FORMATTED_VALUE HTTP/1.1" 200 None
15-Mar-19 12:10:25 DEBUG     https://sheets.googleapis.com:443 "GET /v4/spreadsheets/1lF_AeqkipBki5AIdkDKokqWeB8HEGVrwIpQOWGFk1J8/values/sheet1%21C3?valueRenderOption=FORMATTED_VALUE HTTP/1.1" 200 None
15-Mar-19 12:10:25 DEBUG     https://sheets.googleapis.com:443 "GET /v4/spreadsheets/1lF_AeqkipBki5AIdkDKokqWeB8HEGVrwIpQOWGFk1J8/values/sheet1%21D3?valueRenderOption=FORMATTED_VALUE HTTP/1.1" 200 None
15-Mar-19 12:10:25 DEBUG     https://sheets.googleapis.com:443 "GET /v4/spreadsheets/1lF_AeqkipBki5AIdkDKokqWeB8HEGVrwIpQOWGFk1J8/values/sheet1%21B3?valueRenderOption=FORMATTED_VALUE HTTP/1.1"

BABA BUY
Checking Trend...


15-Mar-19 12:10:30 DEBUG     https://sheets.googleapis.com:443 "GET /v4/spreadsheets/1lF_AeqkipBki5AIdkDKokqWeB8HEGVrwIpQOWGFk1J8/values/sheet1%21A4?valueRenderOption=FORMATTED_VALUE HTTP/1.1" 200 None
15-Mar-19 12:10:30 DEBUG     https://sheets.googleapis.com:443 "GET /v4/spreadsheets/1lF_AeqkipBki5AIdkDKokqWeB8HEGVrwIpQOWGFk1J8/values/sheet1%21B4?valueRenderOption=FORMATTED_VALUE HTTP/1.1" 200 None
15-Mar-19 12:10:30 DEBUG     https://sheets.googleapis.com:443 "GET /v4/spreadsheets/1lF_AeqkipBki5AIdkDKokqWeB8HEGVrwIpQOWGFk1J8/values/sheet1%21C4?valueRenderOption=FORMATTED_VALUE HTTP/1.1" 200 None
15-Mar-19 12:10:30 DEBUG     https://sheets.googleapis.com:443 "GET /v4/spreadsheets/1lF_AeqkipBki5AIdkDKokqWeB8HEGVrwIpQOWGFk1J8/values/sheet1%21D4?valueRenderOption=FORMATTED_VALUE HTTP/1.1" 200 None
15-Mar-19 12:10:30 DEBUG     https://sheets.googleapis.com:443 "GET /v4/spreadsheets/1lF_AeqkipBki5AIdkDKokqWeB8HEGVrwIpQOWGFk1J8/values/sheet1%21B4?valueRenderOption=FORMATTED_VALUE HTTP/1.1"

KeyboardInterrupt: 